In [ ]:
# Import libraries
import sunpy.map
from sunpy.net import Fido, attrs as a
from astropy import units as u
import matplotlib.pyplot as plt
import pandas as pd
import glob

In [ ]:
cd /mnt/data/Normalized_Data/

In [ ]:
import os
print(os.path.exists('/mnt/data/Normalized_Data/aia131_batch1_results_20250710_032122.csv'))

In [ ]:
df1 = pd.read_csv('/mnt/data/Normalized_Data/aia94_batch1_results_20250710_033437.csv')
df2 = pd.read_csv('/mnt/data/Normalized_Data/aia94_batch2_results_20250710_033826.csv')
df3 = pd.read_csv('/mnt/data/Normalized_Data/aia94_batch3_results_20250710_034222.csv')
df4 = pd.read_csv('/mnt/data/Normalized_Data/aia94_batch4_results_20250710_034620.csv')
df5 = pd.read_csv('/mnt/data/Normalized_Data/aia94_batch5_results_20250710_035017.csv')
df6 = pd.read_csv('/mnt/data/Normalized_Data/aia94_batch6_results_20250710_035414.csv')
df7 = pd.read_csv('/mnt/data/Normalized_Data/aia94_batch7_results_20250710_035813.csv')
df8 = pd.read_csv('/mnt/data/Normalized_Data/aia94_batch8_results_20250710_040210.csv')
df9 = pd.read_csv('/mnt/data/Normalized_Data/aia94_batch9_results_20250710_040609.csv')
df10 = pd.read_csv('/mnt/data/Normalized_Data/aia94_batch10_results_20250710_041008.csv')
df11 = pd.read_csv('/mnt/data/Normalized_Data/aia94_batch11_results_20250710_041406.csv')
df12= pd.read_csv('/mnt/data/Normalized_Data/aia94_batch12_results_20250710_041711.csv')

In [ ]:
# Load AIA94 data files and check dfs
aia94_csv_files = sorted(glob.glob('/mnt/data/Normalized_Data/aia94_*_results_*.csv'))
print(f"Found {len(aia94_csv_files)} files to concatenate.")

df_list = []
for file in aia94_csv_files:
    print(f"Loading {file}...")
    df = pd.read_csv(file)
    df_list.append(df)
aia94_combined_df = pd.concat(df_list, ignore_index=True)

#output_path = '/mnt/data/Normalized_Data/aia194_combined_results.csv'
#combined_df.to_csv(output_path, index=False)
aia94_combined_df.head(50)

In [ ]:
aia94_combined_df.info()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
plt.hist(aia94_combined_df['mean'].dropna(), bins=50, color=plt.cm.plasma(0.6))
plt.xlabel('Mean Intensity')
plt.ylabel('Count')
plt.title('Normal Histogram of Mean Intensity of AIA94 Sampling')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
aia94_data = aia94_combined_df['mean'].dropna().values
counts, bin_edges = np.histogram(aia94_data, bins=50)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

## Remove zero counts for log-log
nonzero = counts > 0
plt.figure(figsize=(8,6))
plt.loglog(bin_centers[nonzero], counts[nonzero], marker='o', linestyle='none',
           color=plt.cm.plasma(0.6))
plt.xlabel('Mean Intensity')
plt.ylabel('Count')
plt.title('Log-Log Histogram of Mean Intensity of AIA94 Sampling')
plt.grid(True, which="both", ls="--")
plt.show()

In [ ]:
# Load AIA131 data files and check dfs
aia131_csv_files = sorted(glob.glob('/mnt/data/Normalized_Data/aia131_*_results_*.csv'))
print(f"Found {len(aia131_csv_files)} files to concatenate.")

df_list = []
for file in aia131_csv_files:
    print(f"Loading {file}...")
    df = pd.read_csv(file)
    df_list.append(df)
aia131_combined_df = pd.concat(df_list, ignore_index=True)
#outpt_path = '/mnt/data/Normalized_Data/aia131_combined_results.csv'
#combined_df.to_csv(output_path, index=False)
aia131_combined_df.head(50)

In [ ]:
aia131_combined_df.info()

In [ ]:
aia131_data = aia131_combined_df['mean'].dropna().values
counts, bin_edges = np.histogram(aia131_data, bins=50)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

## Plot for normal histogram
plt.figure(figsize=(8,6))
plt.hist(aia131_combined_df['mean'].dropna(), bins=50, color=plt.cm.plasma(0.6))
plt.xlabel('Mean Intensity')
plt.ylabel('Count')
plt.title('Normal Histogram of Mean Intensity of AIA131 Sampling')
plt.grid(True)
plt.show()

## Remove zero counts for log-log
nonzero = counts > 0
plt.figure(figsize=(8,6))
plt.loglog(bin_centers[nonzero], counts[nonzero], marker='o', linestyle='none',
           color=plt.cm.plasma(0.6))
plt.xlabel('Mean Intensity')
plt.ylabel('Count')
plt.title('Log-Log Histogram of Mean Intensity of AIA131 Sampling')
plt.grid(True, which="both", ls="--")
plt.show()

#### We want to plot the Intensity Histogram across 7/7, 7/11, 7/20, and 8/1 using the preprocesed SDO/AIA data sized at 512 x 512 pixesl.

In [ ]:
pwd

In [ ]:
data_dir = "/mnt/data2/AIA_processed_data/94"
plot_dates = ["2023-07-07", "2023-07-11", "2023-07-20", "2023-08-01"]  # format: YYYY-MM-DD
files = glob.glob(os.path.join(data_dir, "**", "*.fits"), recursive=True)
print(f"Found {len(files)} .fits files")
for f in sorted(files)[:10]:
    print(os.path.basename(f))

In [ ]:
#### We want to plot the Intensity Histogram across 7/7/23, 7/11/23, 7/20/23, and 8/1/23
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import tempfile
import pandas as pd
from datetime import datetime

# Configurable Parameters
data_dir = "/mnt/data2/AIA_processed_data/94"
plot_dates = ["2023-07-07", "2023-07-11", "2023-07-20", "2023-08-01"]  # format: YYYY-MM-DD
saturation_threshold = 16000

## Helper Function to sort files
def find_files_by_date(data_dir, dates):
    all_fits = glob.glob(os.path.join(data_dir, "**", "*.fits"), recursive=True)
    matching_files = [f for f in all_fits if any(date in os.path.basename(f) for date in dates)]
    return sorted(matching_files)

## Create a load and plot function
def load_and_plot(file_path):
    try:
        with fits.open(file_path) as hdul:
            for hdu in hdul:
                data = getattr(hdu, 'data', None)
                if data is not None and data.ndim >= 2:
                    if data.ndim == 3:
                        data = data[0]
                    data = np.nan_to_num(data, nan=0.0, posinf=0.0, neginf=0.0)
                    data[data < 0] = 0
                    pixels = data.flatten()

                    ## Plot Intensity
                    plt.figure(figsize=(6, 5))
                    plt.imshow(np.log10(data + 1), cmap='inferno', origin='lower')
                    plt.colorbar(label='log10(Intensity)')
                    plt.title(f"Intensity Plot\n{os.path.basename(file_path)}")
                    plt.tight_layout()
                    out_name = f"intensity_{os.path.basename(file_path).replace('.fits', '')}.png"
                    plt.savefig(out_name, dpi=200)
                    plt.close()
                    print(f"Saved: {out_name}")

                    ## Compute Stats
                    mean = np.mean(pixels)
                    std = np.std(pixels)
                    min_val = np.min(pixels)
                    max_val = np.max(pixels)
                    p95 = np.percentile(pixels, 95)
                    p99 = np.percentile(pixels, 99)
                    p995 = np.percentile(pixels, 99.5)

                    print(f"Stats for {os.path.basename(file_path)}")
                    print(f"Mean: {mean:.2f}")
                    print(f"Std: {std:.2f}")
                    print(f"Min: {min_val:.2f}, Max: {max_val:.2f}")
                    print(f"95th percentile: {p95:.2f}")
                    print(f"99th percentile: {p99:.2f}")
                    print(f"99.5th percentile: {p995:.2f}")

                    ## Plot Histogram
                    log_bins = np.logspace(np.log10(max(1, pixels.min())), np.log10(pixels.max()), 200)
                    counts, bin_edges = np.histogram(pixels, bins=log_bins)
                    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
                    plt.figure(figsize=(8, 4))
                    plt.bar(np.log10(bin_centers), np.log10(counts + 1), width=0.02, align='center', edgecolor='black')
                    plt.axvline(np.log10(saturation_threshold), color='red', linestyle='--', label='Saturation Threshold')
                    plt.xlabel("log10(Intensity)")
                    plt.ylabel("log10(Count)")
                    plt.title(f"Histogram: {os.path.basename(file_path)}")
                    plt.legend()
                    plt.grid(True)
                    plt.tight_layout()
                    hist_name = f"histogram_{os.path.basename(file_path).replace('.fits', '')}.png"
                    plt.savefig(hist_name, dpi=200)
                    plt.close()
                    print(f"Saved: {hist_name}")
                    return
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
print("Searching for .fits files by date...")
date_files = find_files_by_date(data_dir, plot_dates)
print(f"Found {len(date_files)} matching files.")

for f in date_files:
    load_and_plot(f)
print("Finished generating intensity plots and histograms.")
